In [ ]:
import torch
from transformers import AutoTokenizer, EsmForMaskedLM
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from pyJoules.device.nvidia_device import NvidiaGPUDomain
from pyJoules.handler.pandas_handler import PandasHandler
from pyJoules.energy_meter import measure_energy

from Bio import SeqIO
import pandas as pd
import numpy as np
from time import perf_counter_ns
import matplotlib.pyplot as plt
import gc

SEQ_MAX_LEN = 8096

### CUDA/Torch GPU Setup

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

## Convert FASTA file to pd.DataFrame for easier processing

In [ ]:
def fasta_to_dataframe(fasta_file):
    records = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):
        if len(records) >= 8000:
            break
        # Extract the ID after '>tr'
        id_info = seq_record.id.split('|')[1] # Adjust the index based on your FASTA file format
        sequence = str(seq_record.seq)
        if len(sequence) <= 10000:
            # Generate a random index within the range of the sequence length
            random_index = np.random.randint(0, min(len(sequence), SEQ_MAX_LEN))
            # Replace the character at the random index with '<mask>'
            masked_sequence = sequence[:random_index] + '<mask>' + sequence[random_index+1:]
            
            records.append([id_info, sequence, masked_sequence])
    
    # Create a DataFrame from the records
    df = pd.DataFrame(records, columns=['ID', 'Sequence', 'Masked_Sequence'])
    return df

## Create Dataset

In [ ]:
fasta_df = fasta_to_dataframe("human_protein_seq/uniprotkb_proteome_UP000005640.fasta")

In [ ]:
fasta_df.head()

In [ ]:
fasta_df.shape

## Preparing your model and tokenizer

Now we load our model and tokenizer. If using GPU, use `model.cuda()` to transfer the model to GPU.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = EsmForMaskedLM.from_pretrained("facebook/esm2_t36_3B_UR50D")

In [ ]:
class FastaDataset(Dataset):
    def __init__(self, fasta_df, tokenizer, max_length=512):
        self.fasta_df = fasta_df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.fasta_df)

    def __getitem__(self, idx):
        row = self.fasta_df.iloc[idx]
        inputs = self.tokenizer(row["Masked_Sequence"], max_length=min(len(row["Masked_Sequence"]), self.max_length), truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        mask_position = row["Masked_Sequence"].find('<mask>')
        actual_token = row['Sequence'][mask_position]
        actual_token_id = self.tokenizer.encode(actual_token, add_special_tokens=False)[0]
        target = torch.tensor([actual_token_id] + [0]*33, device=device).unsqueeze(0)
        
        return inputs, target

In [ ]:
# Instantiate the dataset
fasta_dataset = FastaDataset(fasta_df, tokenizer)

# Create a DataLoader
batch_size = 8
dataloader = DataLoader(fasta_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

model.to(device)

# TODO: Revamp following code to implement Datasets and DataLoaders

In [ ]:
print(f"Running test on {fasta_df.shape[0]} sequences.")

gc.disable()

# set up accuracy & latency info - leave this in the same code block so that it resets each time
accuracy = []
latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', "end"])
sequence_length = []
# memory_usage = []
ctr = 0

loss_fn = nn.CrossEntropyLoss()
pandas_handler = PandasHandler()

@measure_energy(domains=[NvidiaGPUDomain(0)], handler=pandas_handler)
def run(accuracy, latency):
    latency_time_stamps = []
    # TIME LOG: start
    latency_time_stamps.append(perf_counter_ns())

    # Tokenize the input and move it to the correct device
    inputs = tokenizer(fasta_row["Masked_Sequence"], max_length=min(len(fasta_row["Masked_Sequence"]), SEQ_MAX_LEN), truncation=True, return_tensors='pt')
    
    # TIME LOG: tokenize
    latency_time_stamps.append(perf_counter_ns())
    
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # TIME LOG: move_to_gpu
    latency_time_stamps.append(perf_counter_ns())
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # TIME LOG: model
    latency_time_stamps.append(perf_counter_ns())
    
    unmasked_text = fasta_row['Sequence']
    masked_text = fasta_row["Masked_Sequence"]
    mask_position = masked_text.find('<mask>')
    actual_token = unmasked_text[mask_position]
    
    # Convert the actual token to its corresponding ID
    actual_token_id = tokenizer.encode(actual_token, add_special_tokens=False)[0]

    # Prepare the target tensor
    target = torch.tensor([actual_token_id] + np.zeros(33, dtype=np.longlong), device=device).unsqueeze(0)
    
    # Calculate the cross-entropy loss
    loss = loss_fn(logits, target)

    # TIME LOG: end
    latency_time_stamps.append(perf_counter_ns())

    # Append the loss to a list or use it as needed
    accuracy.append(loss.item())
        
    # memory_usage.append(torch.cuda.memory_allocated())

    latency.loc[len(latency.index)] = latency_time_stamps

torch.cuda.empty_cache()

for id, fasta_row in fasta_df.iterrows():
    ctr += 1
    if ctr % 100 == 0:
        percentage = "{:.2f}".format(ctr * 100 / fasta_df.shape[0])
        print(f"Processing... {percentage}%")
        
    sequence_length.append(len(fasta_row["Sequence"]))
    
    if len(fasta_row["Sequence"]) > 10000:
        print("Over Mem: " + id)
    
    else:
        torch.cuda.empty_cache()
        # try to save as much memory as possible by reducing fragmentation
        gc.collect()
        
        # load model to GPU
        model.to(device)

        run(accuracy, latency)
        
        # try to remove anything no longer needed on the GPU mem
        gc.collect()
            
gc.enable()
gpu_util_df = pandas_handler.get_dataframe()

Due to memory constraints, sequences over 10k characters in length have been dropped from the dataset for this model; hence only 1 set of graphs are made

### Cross Entropy Loss x Sequence Length

In [ ]:
# Create a scatter plot
plt.scatter(sequence_length, accuracy)

# Label the axes
plt.xlabel('Sequence Length')
plt.ylabel('Cross Entropy Loss')

# Add a title
plt.title('Sequence Length vs. Cross Entropy Loss')

# Display the plot
plt.show()

## Latency/Time Log

In [ ]:
# latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', "end"])
# get_cel = get cross entropy loss
latency_df = pd.DataFrame(columns=['tokenization', 'move_inputs_to_gpu', 'model_process', 'cross_entropy_loss'])

latency_df["tokenization"] = latency["tokenize"] - latency["start"]
latency_df["move_inputs_to_gpu"] = latency["move_to_gpu"] - latency["tokenize"]
latency_df["model_process"] = latency["model"] - latency["move_to_gpu"]
latency_df["cross_entropy_loss"] = latency["end"] - latency["model"]

latency_sums = latency_df.sum()
# Create a bar plot
latency_sums.plot(kind='bar')

# Add title and labels
plt.title('Total Latency per Step')

# Display the plot
plt.show()

### Latency & GPU Usage vs Sequence Length

In [ ]:
gpu_util_df.head()

In [ ]:
# Normalize Duration
duration = gpu_util_df["duration"] * 100000

lat_gpu_slen = {"Latency": duration, "GPU_Utilization": gpu_util_df["nvidia_gpu_0"], "Sequence_Length": sequence_length}
lat_gpu_slen_df = pd.DataFrame(lat_gpu_slen)
lat_gpu_slen_df = lat_gpu_slen_df.sort_values(by = 'Sequence_Length')

# Plot multiple lines
plt.plot(lat_gpu_slen_df["Sequence_Length"], lat_gpu_slen_df["GPU_Utilization"], label='GPU_Utilization')
plt.plot(lat_gpu_slen_df["Sequence_Length"], lat_gpu_slen_df["Latency"], label='Latency')

plt.xlabel('Sequence Length')
plt.ylabel('10ns / Joule')

plt.title("Latency/GPU Util vs Sequence Length")
plt.legend()

plt.show()